In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import re

import requests

import time

bot_token = '5083316257:AAFM1BCJBHF6qLPObG5BLjKOTQwstBKBxas'


In [21]:
searchItem = "Apple AirPods Max"

In [22]:
results = pd.DataFrame(columns= ["Site", "Productname", "Price", "Link"])
message = ""

In [23]:
chrome_options = Options()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--start-maximized");
chrome_options.add_argument("disable-infobars");
chrome_options.add_argument('--profile-directory=Default')
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--disable-plugins-discovery")

chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

chrome_options.add_argument("window-size=1280,800")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")

In [24]:
def search_mediamarkt():
    site = "https://www.mediamarkt.de/"

    driver = webdriver.Chrome(r"Webdriver\chromedriver.exe", chrome_options=chrome_options)
    driver.delete_all_cookies()
    driver.get(site)
    
    try:
        searchBar = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".StyledTextInput-p8hv3c-1.gYopDH")))
        searchBar.clear()
        searchBar.send_keys(searchItem)
        searchBar.send_keys(Keys.RETURN)

        title = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".BaseTypo-sc-1jga2g7-0.izkVco.StyledInfoTypo-sc-1jga2g7-1.doYUxh"))).text

        price = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".StyledTooltipWrapper-sc-1rht449-0.bCtFKC"))).text
        price = int(re.search('[0-9]+', price).group())

        link = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".StyledLinkRouter-sc-1drhx1h-2.cOmqtX.StyledLink-xw7f64-0.cWtdqC"))).get_attribute("href")

        return({"Site": "Media Markt", "Productname": title, "Price": price, "Link":link})
    
    except: 
        pass
    
    driver.close()

In [25]:
def search_amazon():
    site = "http://www.amazon.de"
    
    driver = webdriver.Chrome(r"Webdriver\chromedriver.exe", chrome_options=chrome_options)
    driver.delete_all_cookies()
    driver.get(site)

    try:
        searchBar = driver.find_element(By.ID, "twotabsearchtextbox")
        searchBar.clear()
        searchBar.send_keys(searchItem)
        searchBar.send_keys(Keys.RETURN)

        resultsAmazon = WebDriverWait(driver,10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".s-result-item.s-asin.sg-col-0-of-12.sg-col-16-of-20.AdHolder.sg-col.s-widget-spacing-small.sg-col-12-of-16")))
        result = resultsAmazon[0]

        title = WebDriverWait(result,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".a-link-normal.s-link-style.a-text-normal"))).text

        price = WebDriverWait(result,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".a-price-whole"))).text
        price = int(re.search('[0-9]+', price).group())

        link = WebDriverWait(result,10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".a-link-normal.s-link-style.a-text-normal"))).get_attribute("href")

        return({"Site": "Amazon", "Productname": title, "Price": price, "Link":link})

    except:
        pass
    
    driver.close()

In [26]:
def airpods_apple():
    site = "https://www.apple.com/de/shop/buy-airpods/airpods-max/space-grau"

    driver = webdriver.Chrome(r"Webdriver\chromedriver.exe")
    driver.delete_all_cookies()
    driver.get(site)

    time.sleep(3)
    price = WebDriverWait(driver,100).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".rc-prices-fullprice"))).text
    price = int(re.search('[0-9]+', price).group())

    return({"Site": "Apple", "Productname": "AirPods Max", "Price": price, "Link":site})



In [33]:
results = results.append(search_amazon(), ignore_index=True)
results = results.append(search_mediamarkt(), ignore_index=True)
results = results.append(airpods_apple(), ignore_index=True)

C:\Users\NICOLA~1\AppData\Local\Temp/ipykernel_7768/3208368711.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"Webdriver\chromedriver.exe", chrome_options=chrome_options)
C:\Users\NICOLA~1\AppData\Local\Temp/ipykernel_7768/3208368711.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(r"Webdriver\chromedriver.exe", chrome_options=chrome_options)


In [34]:
cheapest = results.loc[results['Price'] == min(results.Price)].iloc[0]
expensive = results.loc[results['Price'] == max(results.Price)].iloc[0]
discount = 100 - ((100 / (int(expensive.Price))) * int(cheapest.Price))

message = "Todays cheapest price is {}€, form {} ({}). It is {}% cheaper than the expensivest website ({})".format(int(cheapest.Price), cheapest.Site[0], cheapest.Link[0], discount, expensive.Link[0])


In [30]:
def telegram_bot_sendtext(bot_message, bot_token):
    params = {"chat_id": "5081325726", "text":bot_message}
    
    url = "https://api.telegram.org/bot{}/sendMessage".format(bot_token)
    message = requests.post(url, params=params)
    return(message)

In [35]:
telegram_bot_sendtext(message,bot_token)

<Response [200]>